# What is the density of mCherry cells?

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
project = "becalia_rabies_barseq"
mouse = "BRAC8498.3e"

ara_projection = "flatmap_dorsal"
atlas_size = 10

# Load data

mCherry cells first

In [ ]:
import iss_analysis as issa

mcherry_cells = issa.io.get_mcherry_cells(
    project, mouse, verbose=True, which="curated", prefix="mCherry_1"
)

In [ ]:
from brisc.manuscript_analysis.overview_image import run_plot_overview

fig = run_plot_overview(
    channel_colors=([0, 1, 1], [1, 0, 1], [1, 0, 0], [0, 1, 0]),
    vmin=[15, 65, 63, 120],
    vmax=[2200, 1000, 1150, 1200],
)

In [ ]:
from brisc.manuscript_analysis.overview_image import plot_rabies_raw
import matplotlib.pyplot as plt

fig, ax = plt.subplots(dpi=200)
plot_rabies_raw(
    ax,
    "/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8498.3e/chamber_10/figures/round_overviews/chamber_10_roi_03_barcode_round_1_1_channels_0_1_2_3.png",
    crop_top=450,
    crop_bottom=650,
    crop_left=150,
    crop_right=50,
)

Then all cells

In [ ]:
import flexiznam as flz
import pandas as pd

df_file = flz.get_processed_path(
    "becalia_rabies_barseq/BRAC8498.3e/analysis/cell_barcode_df.pkl"
)

full_df = pd.read_pickle(df_file)
print(f"Loaded {len(full_df)} cells")
barcoded_cells = full_df.query("main_barcode.notna()")
print(f"Found {len(barcoded_cells)} barcoded cells")
starter_cells_df = barcoded_cells.query("is_starter == True")
starter_barcode = {}
for i, row in starter_cells_df.iterrows():
    for bc in row["all_barcodes"]:
        starter_barcode.setdefault(bc, []).append(i)
print(
    f"Found {len(starter_barcode)} unique starter barcodes in {len(starter_cells_df)} starter cells"
)

# Do brighter mCherry cells have more presynaptic cells?

Look at the number of presynpatic for all starter

In [ ]:
mcherry_cells.head()

In [ ]:
# Add mcherry intensity to the starter_df
import numpy as np

mcherry_cells.set_index("mcherry_uid", inplace=True, drop=False)
starter_cells_df = starter_cells_df.copy()
starter_cells_df["mcherry_intensity"] = starter_cells_df["mcherry_uid"].map(
    lambda x: mcherry_cells.loc[x, "intensity_mean-1"]
)

In [ ]:
# BC present in only 1 starter:
single_starter_barcodes = {k: v for k, v in starter_barcode.items() if len(v) == 1}
print(f"Found {len(single_starter_barcodes)} barcodes present in only 1 starter cell")
mcherry_cells.set_index("mcherry_uid", inplace=True, drop=False)
mcherry_cells["is_starter"] = False
mcherry_cells["n_presynaptic"] = np.nan

exploded = barcoded_cells.all_barcodes.explode()
for bc, starter in single_starter_barcodes.items():
    s_df = starter_cells_df.loc[starter[0]]
    mch = s_df.mcherry_uid
    cell_with_bc = exploded[exploded == bc].copy()
    mcherry_cells.loc[mch, "is_starter"] = True
    mcherry_cells.loc[mch, "n_presynaptic"] = len(cell_with_bc)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

valid = mcherry_cells.query("is_starter == True")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for iax, w in enumerate(["intensity_max-0", "intensity_mean-0", "intensity_ratio"]):
    ax = axes[iax]
    ax.set(xscale="log", yscale="log")
    ax.scatter(valid[w], valid["n_presynaptic"], alpha=0.5)
    ax.set_xlabel(w)
    ax.set_ylabel("Number of presynaptic cells + 1 (starter)")
fig.tight_layout

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
sc = ax.scatter(
    valid["intensity_max-0"],
    valid["intensity_ratio"],
    c=np.log(valid["n_presynaptic"]),
    alpha=0.5,
)
ax.set(xscale="log", yscale="log")
cb = plt.colorbar(sc)
cb.set_label("log(Number of presynaptic cells + 1)")
ax.set_xlabel("Max intensity")
ax.set_ylabel("Intensity ratio")

In [ ]:
(mcherry_cells["intensity_mean-0"] > 200).value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(111)
_ = ax.hist(mcherry_cells["intensity_mean-0"], bins=100)
ax.set_xlabel("mCherry intensity")
ax.set_ylabel("Number of cells")

In [ ]:
plt.scatter(
    mcherry_cells["intensity_mean-0"], mcherry_cells["intensity_mean-1"], alpha=0.5
)
plt.xscale("log")
plt.yscale("log")
plt.ylabel("Intensity mean 1")
plt.xlabel("Intensity mean 0")

In [ ]:
import numpy as np
import iss_preprocess as issp

cols = ["ara_x", "ara_y", "ara_z", "area_id", "area_acronym"]
for col in cols:
    mcherry_cells[col] = np.nan

for (chamber, roi), cells in mcherry_cells.groupby(["chamber", "roi"]):
    print(chamber, roi)
    ara_infos_spots = issp.pipeline.ara_registration.spots_ara_infos(
        data_path=f"{project}/{mouse}/{chamber}",
        spots=cells,
        roi=roi,
        atlas_size=atlas_size,
        acronyms=True,
        inplace=False,
        full_scale_coordinates=False,
        reload=True,
        verbose=False,
    )
    mcherry_cells.loc[cells.index, cols] = ara_infos_spots[cols]

In [ ]:
# Get the atlas background
import ccf_streamlines.projection as ccfproj
import bg_atlasapi as bga
import flexiznam as flz
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from cricksaw_analysis import atlas_utils
from scipy.ndimage import binary_dilation


# Get atlas via brainglobe
atlas_size = int(atlas_size)
atlas = bga.bg_atlas.BrainGlobeAtlas(f"allen_mouse_{atlas_size}um")

# flat map via ccf_streamlines
project_folder = Path("/nemo/lab/znamenskiyp/home/shared/")
ccf_streamlines_folder = project_folder / "resources" / "ccf_streamlines"
# Get a projector object
ccf_coord_proj = ccfproj.IsocortexCoordinateProjector(
    projection_file=ccf_streamlines_folder / f"{ara_projection}.h5",
    surface_paths_file=ccf_streamlines_folder / "surface_paths_10_v3.h5",
    closest_surface_voxel_reference_file=ccf_streamlines_folder
    / "closest_surface_voxel_lookup.h5",
    streamline_layer_thickness_file=ccf_streamlines_folder / "cortical_layers_10_v2.h5",
)

In [ ]:
midline = int(np.where(np.diff(atlas.hemispheres[0, 0, :]) != 0)[0])

# We can highlight some areas, we define that has a dict with the area name as key
# and the color as value
area_colors = {
    "VISp": "green",
    "VISpm": "blue",
    "VISam": "red",
    "VISal": "orange",
    "VISl": "purple",
    "VISrl": "yellow",
    "VISa": "cyan",
    "VISli": "pink",
    "VISpor": "bisque",
    "AUDp": "teal",
    "AUDd": "magenta",
    "AUDpo": "brown",
    "AUDv": "orchid",
    "TEa": "royalblue",
}

ctx_table = atlas_utils.create_ctx_table(atlas=atlas)


def find_l1_pixels(atlas_img):
    """Return the coordinates of the surface pixel in cortex"""
    layer1_ids = ctx_table[ctx_table["layer"] == "1"].id.values
    layer1_mask = np.isin(atlas_img, layer1_ids)
    return np.where(layer1_mask)


def plot_line(flat_ax, atlas, cor_plane=800, hemisphere="right", **kwargs):
    line_kwargs = dict(color="grey", ls="--")
    if hemisphere == "left":
        cor_atlas = atlas.annotation[cor_plane, :, :midline]
    elif hemisphere == "right":
        cor_atlas = atlas.annotation[cor_plane, :, midline:]
    else:
        cor_atlas = atlas.annotation[cor_plane, :, :]

    line_kwargs.update(kwargs)
    dv, ml = find_l1_pixels(cor_atlas)
    if hemisphere == "right":
        ml = midline + ml
    coor_plane_coords = np.vstack((np.array([cor_plane] * len(dv)), dv, ml)).T
    subsample = max(1, len(coor_plane_coords) // 100)
    coor_plane_coords = coor_plane_coords[::subsample]
    flat_coords = ccf_coord_proj.project_coordinates(
        coor_plane_coords * atlas_size,
        drop_voxels_outside_view_streamlines=False,
        view_space_for_other_hemisphere=ara_projection,
        hemisphere=hemisphere,
    )
    # subsample to keep about 10 points
    ordering = np.argsort(flat_coords[:, 0])
    subsample = max(1, len(coor_plane_coords) // 100)
    flat_coords = flat_coords[ordering][::subsample]
    flat_ax.plot(flat_coords[:, 0], flat_coords[:, 1], **line_kwargs)


def plot_background(
    cor_ax,
    flat_ax,
    cor_plane=800,
    hemisphere="right",
    area_colors=area_colors,
    lines_to_add=None,
    alpha=0.1,
    label_areas=True,
):
    # Coronal view
    cor_atlas = atlas_utils.plot_coronal_view(
        cor_ax,
        cor_plane,
        atlas,
        hemisphere=hemisphere,
        area_colors=area_colors,
        alpha=alpha,
    )
    atlas_utils.plot_flatmap(
        flat_ax,
        hemisphere=hemisphere,
        area_colors=area_colors,
        alpha=alpha,
        ccf_streamlines_folder=None,
        label_areas=label_areas,
    )

    # add a line to show the plane
    plot_line(flat_ax, atlas, cor_plane=cor_plane, hemisphere=hemisphere, lw=1)
    if lines_to_add is not None:
        for line in lines_to_add:
            flat_ax.plot(*line, color="black", lw=1)

    flat_ax.set_aspect("equal")
    flat_ax.set_ylim(1380, 600)
    flat_ax.set_xlim(50, 1150)
    cor_ax.set_xlim(-10, 600)
    cor_ax.set_ylim(500, -10)


def plot_spots(cor_ax, flat_ax, df, **kwargs):
    cor_coors = np.vstack(
        [
            df["ara_z"].values * 1000 / atlas_size - midline,
            df["ara_y"].values * 1000 / atlas_size,
        ]
    )
    cor_ax.scatter(*cor_coors, **kwargs)
    flat_coords = df[["ara_x", "ara_y", "ara_z"]].values
    bad = np.any(flat_coords < 0, axis=1)
    # also remove NaN
    bad = np.logical_or(bad, np.any(np.isnan(flat_coords), axis=1))
    if np.any(bad):
        print(f"Found {np.sum(bad)} bad coordinates")
        flat_coords = flat_coords[~bad]
    flat_coords = ccf_coord_proj.project_coordinates(
        flat_coords * 1000,
        drop_voxels_outside_view_streamlines=False,
        view_space_for_other_hemisphere=ara_projection,
        hemisphere="right",
    )

    flat_ax.scatter(flat_coords[:, 0], flat_coords[:, 1], **kwargs)
    return cor_coors, flat_coords


fig = plt.figure(figsize=(20, 15))
cor_ax = fig.add_subplot(121)
flat_ax = fig.add_subplot(122)
plot_background(cor_ax, flat_ax, cor_plane=880, label_areas=False)
cor_coors, flat_coords = plot_spots(
    cor_ax, flat_ax, mcherry_cells, color="royalblue", s=10, alpha=0.3, ec="none"
)
cor_coors_starter, flat_coords_starter = plot_spots(
    cor_ax,
    flat_ax,
    starter_cells_df,
    cmap="Reds",
    c=np.log(starter_cells_df["mcherry_intensity"]),
    s=20,
    alpha=0.5,
    ec="none",
)
cor_ax.set_xlim([50, 500])
cor_ax.set_ylim([250, 20])
flat_ax.set_xlim([400, 800])
flat_ax.set_ylim([1200, 900])
flat_ax.axis("off")
_ = cor_ax.axis("off")
fig.tight_layout()
save_folder = flz.get_processed_path("becalia_rabies_barseq/BRAC8498.3e/analysis")
fig.savefig(save_folder / "starter_cell_distribution.png", dpi=300)

In [ ]:
from iss_preprocess
def get_mcherry_tile(data_path, roi, ara_x, ara_y, ara_z):
        
    coords = load_coordinate_image(data_path, roi, full_scale=False)
    metadata = load_registration_reference_metadata(data_path, roi)


In [ ]:
coords = np.array(mcherry_cells[["ara_x", "ara_y", "ara_z"]].values) * 1000


cube_size = 500
# make a 3d histogram
n_bins = [np.arange(0, atlas.shape[i] * atlas_size, cube_size) for i in range(3)]
hist, edges = np.histogramdd(coords, bins=n_bins)

In [ ]:
max_index = np.unravel_index(hist.argmax(), hist.shape)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.axis("off")
ax.set_aspect("equal")
volume = (cube_size / 1000) ** 3
density = hist / volume
density_max = density.max()
density[density == 0] = np.nan
im = ax.imshow(density[max_index[0]], cmap="viridis", vmin=0, vmax=density_max)
plt.colorbar(im, ax=ax)